# Using JAX for faster sampling

(c) Thomas Wiecki, 2020

*Note: These samplers are still experimental.*

Using the new Aesara JAX linker that Brandon Willard has developed, we can compile PyMC models to JAX without any change to the PyMC code base or any user-level code changes. The way this works is that we take our Aesara graph built by PyMC and then translate it to JAX primitives. 

Using our Python samplers, this is still a bit slower than the C-code generated by default Aesara.

However, things get really interesting when we also express our samplers in JAX. Here we have used the JAX samplers by NumPyro or TFP. This combining of the samplers was done by [Junpeng Lao](https://twitter.com/junpenglao). 

The reason this is so much faster is that while before in PyMC, only the logp evaluation was compiled while the samplers where still coded in Python, so for every loop we went back from C to Python. With this approach, the model *and* the sampler are JIT-compiled by JAX and there is no more Python overhead during the whole sampling run. This way we also get sampling on GPUs or TPUs for free.

This NB requires the master of [Aesara-PyMC](https://github.com/pymc-devs/Aesara-PyMC), the [pymcjax branch of PyMC](https://github.com/pymc-devs/pymc/tree/pymcjax), as well as JAX, TFP-nightly and numpyro.

This is all still highly experimental but extremely promising and just plain amazing.

As an example we'll use the classic Radon hierarchical model. Note that this model is still very small, I would expect much more massive speed-ups with larger models.

In [1]:
import warnings

import aesara
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import pymc.sampling_jax

print(f"Running on PyMC v{pm.__version__}")

Running on PyMC v3.10.0


/Users/CloudChaoszero/opt/anaconda3/envs/pymc-dev/lib/python3.8/site-packages/pymc/sampling_jax.py:22: UserWarning: This module is experimental.
  warnings.warn("This module is experimental.")


In [2]:
warnings.filterwarnings("ignore")

In [3]:
%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

In [4]:
data = pd.read_csv(pm.get_data("radon.csv"))
data["log_radon"] = data["log_radon"].astype(aesara.config.floatX)
county_names = data.county.unique()
county_idx = data.county_code.values.astype("int32")

n_counties = len(data.county.unique())

Unchanged PyMC model specification:

In [5]:
with pm.Model() as hierarchical_model:
    # Hyperpriors for group nodes
    mu_a = pm.Normal("mu_a", mu=0.0, sigma=100.0)
    sigma_a = pm.HalfNormal("sigma_a", 5.0)
    mu_b = pm.Normal("mu_b", mu=0.0, sigma=100.0)
    sigma_b = pm.HalfNormal("sigma_b", 5.0)

    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    a = pm.Normal("a", mu=mu_a, sigma=sigma_a, shape=n_counties)
    # Intercept for each county, distributed around group mean mu_a
    b = pm.Normal("b", mu=mu_b, sigma=sigma_b, shape=n_counties)

    # Model error
    eps = pm.HalfCauchy("eps", 5.0)

    radon_est = a[county_idx] + b[county_idx] * data.floor.values

    # Data likelihood
    radon_like = pm.Normal("radon_like", mu=radon_est, sigma=eps, observed=data.log_radon)

## Sampling using our old Python NUTS sampler

In [6]:
%%time
with hierarchical_model:
    hierarchical_trace = pm.sample(
        2000, tune=2000, target_accept=0.9, compute_convergence_checks=False
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO (aesara.gof.compilelock): Waiting for existing lock by process '74842' (I am process '74843')
INFO (aesara.gof.compilelock): To manually release the lock, delete /Users/CloudChaoszero/.aesara/compiledir_macOS-10.16-x86_64-i386-64bit-i386-3.8.5-64/lock_dir
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [eps, b, a, sigma_b, mu_b, sigma_a, mu_a]


Sampling 2 chains for 2_000 tune and 2_000 draw iterations (4_000 + 4_000 draws total) took 115 seconds.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.


CPU times: user 11.2 s, sys: 2.17 s, total: 13.3 s
Wall time: 2min 45s


## Sampling using JAX TFP NUTS sampler

In [10]:
%%time
# Inference button (TM)!
with hierarchical_model:
    hierarchical_trace_jax = pm.sampling_jax.sample_numpyro_nuts(2000, tune=2000, target_accept=0.9)

Compiling...
Compilation + sampling time =  0 days 00:00:24.698649
CPU times: user 28.7 s, sys: 1.79 s, total: 30.5 s
Wall time: 25.3 s


In [11]:
print(f"Speed-up = {180 / 24}x")

Speed-up = 7.5x


In [12]:
az.plot_trace(
    hierarchical_trace_jax,
    var_names=["mu_a", "mu_b", "sigma_a_log__", "sigma_b_log__", "eps_log__"],
);

In [13]:
az.plot_trace(hierarchical_trace_jax, var_names=["a"], coords={"a_dim_0": range(5)});

In [14]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Sun Jan 17 2021

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

arviz     : 0.10.0
aesara    : 1.0.14
numpy     : 1.19.2
pymc     : 3.10.0
pandas    : 1.1.5
matplotlib: 3.3.3

Watermark: 2.1.0

